TODO:

* Mudar date para fullDate, day, dayOfWeek, month, bimonthly, quarter, fourMonths, year, decade

* we have synthetically associated a city, choosing randomly a city among the one hundred with largest populations. Therefore, the analytic queries related to geographic data have a synthetic character, but they are important to exemplify their potential in a real-world scenario.

* Testar para ver se eu consigo retirar a localizção a partir do texto

* Ajeitar a tabela de cities para ter o que é necessário apenas

* Trocar o dataset para o outro porque nele tem authors e linkSource

* Rodar o flairnovamente? acho que pelo index já consigo pegar sendo o mesmo

Esquema estrela
===
date (dateID, fullDate, day, dayOfWeek, month, bimonthly, quarter, fourMonths, year, decade)

location (locationID, city, state, country, population, latitude, longitude)

content (contentID, title, summary, authors, linkSource, category, sentiment)

event (dateID, locationID, contentID, NumberNegativeNews, NumberPositiveNews, totalQuantity)

==

Duvidas:

* O que é fullDate? 
* O que é o fourMonths?
* É country mesmo ou county?


In [2]:
import pandas as pd

politics = pd.read_csv("pol.csv")
politics = politics.drop(['Unnamed: 0','Unnamed: 0.1'],axis=1)
cities = pd.read_csv("uscities.csv")

In [3]:
politics.head()

,index,headline,short_description,category,date,sentiment
0,21,Biden Says U.S. Forces Would Defend Taiwan If ...,President issues vow as tensions with China rise.,POLITICS,2022-09-19,neg
1,24,‘Beautiful And Sad At The Same Time’: Ukrainia...,An annual celebration took on a different feel...,POLITICS,2022-09-19,pos
2,30,Biden Says Queen's Death Left 'Giant Hole' For...,"U.S. President Joe Biden, in London for the fu...",POLITICS,2022-09-18,neg
3,40,Bill To Help Afghans Who Escaped Taliban Faces...,Republican outrage over the shoddy U.S. withdr...,POLITICS,2022-09-16,pos
4,44,Mark Meadows Complies With Justice Dept. Subpo...,The former White House chief of staff has turn...,POLITICS,2022-09-15,pos


In [4]:
politics_sent = politics.drop(['headline', 'short_description','category','date'],axis='columns')


In [5]:
politics_sent.head()

,index,sentiment
0,21,neg
1,24,pos
2,30,neg
3,40,pos
4,44,pos


In [ ]:
### Raw

import pandas as pd

news_data = pd.read_json("news.json",orient="records",lines=True)

politics_raw = news_data[news_data['category']=="POLITICS"]

politics_cleaned = politics_raw.dropna()

politics_cleaned = politics_cleaned[politics_cleaned['authors']!=""]

In [ ]:
# politics_merged = politics_cleaned.join(politics_sent.set_index('index'), on=politics_cleaned.index)
# politics_final = politics_merged.drop(["key_0"],axis='columns')

In [8]:
# Date

from datetime import datetime

from datetime import datetime

def extract_periods(date: datetime):
    day = date.day
    year = date.year
    month = date.month
    
    bimonthly = (month - 1) // 2 + 1

    quarter = (month - 1) // 3 + 1

    four_months = (month - 1) // 4 + 1

    decade = (year // 10) * 10

    weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday","Sunday"]
    weekday = weekdays[date.weekday()]

    return {
        "fullDate":str(date.__str__()[:10]),
        "day":day,
        "dayOfWeek":weekday,
        "month": month,
        "bimonthly": bimonthly,
        "quarter": quarter,
        "fourMonths": four_months,
        "year": year,
        "decade": decade,
    }

teste = datetime.fromisoformat('2011-11-04')
print(list(extract_periods(teste).values()))

['2011-11-04', 4, 'Friday', 11, 6, 4, 3, 2011, 2010]


## Date table

To each date in the news table we need to have

In [9]:
date_df = pd.DataFrame(columns=["fullDate","day","dayOfWeek", "month", "bimonthly", "quarter", "fourMonths", "year", "decade"])

for date in politics_final['date'].unique():
    extracted_date = extract_periods(date)
    date_df.loc[len(date_df)] = extracted_date.values()

date_df = date_df.drop_duplicates(subset="fullDate")

date_df.head(n=50)

,fullDate,day,dayOfWeek,month,bimonthly,quarter,fourMonths,year,decade
0,2022-09-19,19,Monday,9,5,3,3,2022,2020
1,2022-09-18,18,Sunday,9,5,3,3,2022,2020
2,2022-09-16,16,Friday,9,5,3,3,2022,2020
3,2022-09-15,15,Thursday,9,5,3,3,2022,2020
4,2022-09-14,14,Wednesday,9,5,3,3,2022,2020
5,2022-09-13,13,Tuesday,9,5,3,3,2022,2020
6,2022-09-12,12,Monday,9,5,3,3,2022,2020
7,2022-09-11,11,Sunday,9,5,3,3,2022,2020
8,2022-09-10,10,Saturday,9,5,3,3,2022,2020
9,2022-09-08,8,Thursday,9,5,3,3,2022,2020


# Cities

In [10]:
cities.head()

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,New York,New York,NY,New York,36061,New York,40.6943,-73.9249,18713220,10715,polygon,False,True,America/New_York,1,11229 11226 11225 11224 11222 11221 11220 1138...,1840034016
1,Los Angeles,Los Angeles,CA,California,6037,Los Angeles,34.1139,-118.4068,12750807,3276,polygon,False,True,America/Los_Angeles,1,90291 90293 90292 91316 91311 90037 90031 9000...,1840020491
2,Chicago,Chicago,IL,Illinois,17031,Cook,41.8373,-87.6862,8604203,4574,polygon,False,True,America/Chicago,1,60018 60649 60641 60640 60643 60642 60645 6064...,1840000494
3,Miami,Miami,FL,Florida,12086,Miami-Dade,25.7839,-80.2102,6445545,5019,polygon,False,True,America/New_York,1,33129 33125 33126 33127 33128 33149 33144 3314...,1840015149
4,Dallas,Dallas,TX,Texas,48113,Dallas,32.7936,-96.7662,5743938,1526,polygon,False,True,America/Chicago,1,75287 75098 75233 75254 75251 75252 75253 7503...,1840019440


In [11]:
#city, state, country, population, latitude, longitude 

cities_filtered = cities[["city","state_name","population","lat","lng"]]
cities_filtered['country'] = "United States"
cities_filtered

/tmp/ipykernel_1416422/3663940369.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cities_filtered['country'] = "United States"


,city,state_name,population,lat,lng,country
0,New York,New York,18713220,40.6943,-73.9249,United States
1,Los Angeles,California,12750807,34.1139,-118.4068,United States
2,Chicago,Illinois,8604203,41.8373,-87.6862,United States
3,Miami,Florida,6445545,25.7839,-80.2102,United States
4,Dallas,Texas,5743938,32.7936,-96.7662,United States
...,...,...,...,...,...,...
28333,Gross,Nebraska,2,42.9461,-98.5697,United States
28334,Lotsee,Oklahoma,2,36.1334,-96.2091,United States
28335,The Ranch,Minnesota,2,47.3198,-95.6952,United States
28336,Shamrock,Oklahoma,2,35.9113,-96.5772,United States


In [12]:
cities_filtered.to_csv("cities.csv")

In [13]:
top100_cities = cities_filtered.sort_values(by="population",ascending=False)
top100_cities = top100_cities[:100]
top100_cities = top100_cities.rename({"state_name":"state","lat":"latitude","lng":"longitude"})
top100_cities

,city,state_name,population,lat,lng,country
0,New York,New York,18713220,40.6943,-73.9249,United States
1,Los Angeles,California,12750807,34.1139,-118.4068,United States
2,Chicago,Illinois,8604203,41.8373,-87.6862,United States
3,Miami,Florida,6445545,25.7839,-80.2102,United States
4,Dallas,Texas,5743938,32.7936,-96.7662,United States
...,...,...,...,...,...,...
95,Denton,Texas,457177,33.2176,-97.1419,United States
96,Madison,Wisconsin,447245,43.0826,-89.3931,United States
97,Reno,Nevada,445020,39.5497,-119.8483,United States
98,Harrisburg,Pennsylvania,442289,40.2752,-76.8843,United States


## Content Table

In [14]:
politics_final.head()

,link,headline,category,short_description,authors,date,sentiment
24,https://www.huffpost.com/entry/ukraine-festiva...,‘Beautiful And Sad At The Same Time’: Ukrainia...,POLITICS,An annual celebration took on a different feel...,Jonathan Nicholson,2022-09-19,pos
30,https://www.huffpost.com/entry/europe-britain-...,Biden Says Queen's Death Left 'Giant Hole' For...,POLITICS,"U.S. President Joe Biden, in London for the fu...","Darlene Superville, AP",2022-09-18,neg
40,https://www.huffpost.com/entry/afghan-adjustme...,Bill To Help Afghans Who Escaped Taliban Faces...,POLITICS,Republican outrage over the shoddy U.S. withdr...,Hamed Ahmadi and Arthur Delaney,2022-09-16,pos
44,https://www.huffpost.com/entry/capitol-riot-in...,Mark Meadows Complies With Justice Dept. Subpo...,POLITICS,The former White House chief of staff has turn...,"ERIC TUCKER, AP",2022-09-15,pos
54,https://www.huffpost.com/entry/seth-magaziner-...,Democrats Nominate Seth Magaziner In Key Rhode...,POLITICS,The state's general treasurer is slated to fac...,Daniel Marans,2022-09-14,pos


In [15]:
content_table = politics_final.rename(columns={"link":"linkSource","headline":"title","short_description":"summary"})
# content_table = content_table.drop(["date"],axis="columns")
content_table = content_table.reset_index(drop=True)

In [16]:
content_table.head(n=50)

,linkSource,title,category,summary,authors,date,sentiment
0,https://www.huffpost.com/entry/ukraine-festiva...,‘Beautiful And Sad At The Same Time’: Ukrainia...,POLITICS,An annual celebration took on a different feel...,Jonathan Nicholson,2022-09-19,pos
1,https://www.huffpost.com/entry/europe-britain-...,Biden Says Queen's Death Left 'Giant Hole' For...,POLITICS,"U.S. President Joe Biden, in London for the fu...","Darlene Superville, AP",2022-09-18,neg
2,https://www.huffpost.com/entry/afghan-adjustme...,Bill To Help Afghans Who Escaped Taliban Faces...,POLITICS,Republican outrage over the shoddy U.S. withdr...,Hamed Ahmadi and Arthur Delaney,2022-09-16,pos
3,https://www.huffpost.com/entry/capitol-riot-in...,Mark Meadows Complies With Justice Dept. Subpo...,POLITICS,The former White House chief of staff has turn...,"ERIC TUCKER, AP",2022-09-15,pos
4,https://www.huffpost.com/entry/seth-magaziner-...,Democrats Nominate Seth Magaziner In Key Rhode...,POLITICS,The state's general treasurer is slated to fac...,Daniel Marans,2022-09-14,pos
5,https://www.huffpost.com/entry/biden-cancer-mo...,Joe Biden Urges National Unity In Speech On Re...,POLITICS,"""Cancer does not discriminate red and blue,"" t...",Nick Visser,2022-09-13,pos
6,https://www.huffpost.com/entry/tim-scott-senat...,Sen. Tim Scott Downplays Electability Concerns...,POLITICS,"""Who we have on the field is who we’re gonna p...",Marita Vlachou,2022-09-12,pos
7,https://www.huffpost.com/entry/democratic-prim...,Electability Is On The Ballot In Key Rhode Isl...,POLITICS,Democrats in the state’s more moderate House d...,Daniel Marans,2022-09-12,pos
8,https://www.huffpost.com/entry/biden-moonshot-...,"In A Nod To JFK, Joe Biden Pushing 'Moonshot' ...",POLITICS,Biden plans to draw attention to a new federal...,"Zeke Miller and Carla K. Johnson, AP",2022-09-12,pos
9,https://www.huffpost.com/entry/sept-11-anniver...,"Biden Honors 9/11 Victims, Vows Commitment To ...",POLITICS,President Joe Biden has marked the 21st annive...,"COLLEEN LONG and AAMER MADHANI, AP",2022-09-11,pos


# Event

event (dateID, locationID, contentID, NumberNegativeNews, NumberPositiveNews, totalQuantity)


In [17]:
import random

event_table = pd.DataFrame(columns=["dateID","locationID","contentID","NumberNegativeNews","NumberPositiveNews","totalQuantity"])

for index,event in content_table.iterrows():  
    date_id = date_df.loc[date_df['fullDate']== str(event.date)[:10]].index[0]
    location_id = random.randrange(0,len(cities))
    content_id = index
    number_negative = 1 if event.sentiment == "neg" else 0
    number_positive = 1 if event.sentiment == "pos" else 0
    total_qnt = 1 if number_negative==1 or number_positive == 1 else 0
    
    new_row = [date_id,location_id,content_id,number_negative,number_positive,total_qnt]
    event_table.loc[len(event_table)] = new_row


In [18]:
event_table.head(n=50)

,dateID,locationID,contentID,NumberNegativeNews,NumberPositiveNews,totalQuantity
0,0,10805,0,0,1,1
1,1,11867,1,1,0,1
2,2,9860,2,0,1,1
3,3,8870,3,0,1,1
4,4,9867,4,0,1,1
5,5,19252,5,0,1,1
6,6,19932,6,0,1,1
7,6,15998,7,0,1,1
8,6,4149,8,0,1,1
9,7,10759,9,0,1,1


In [20]:
event_table.to_csv("event.csv",index=False)